In [77]:
# %pip install transformers googletrans==4.0.0-rc1 nltk h5py json urllib
# %pip install transformers deep-translator nltk h5py json urllib
%pip install translators

import os
import json
from transformers import BertTokenizer
import translators as ts
import urllib.request

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from tqdm import tqdm
import time
import random


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [78]:
data_folder = os.path.join(os.getcwd(), "data")
os.makedirs(data_folder, exist_ok=True)
cap_path = os.path.join(data_folder, "captions.json")

if not os.path.exists(cap_path):
    cap_url = "https://data.mendeley.com/public-files/datasets/rxxch9vw59/files/9b3e789a-5a5c-48b3-8a2c-2c91e9307c2a/file_downloaded"
    req = urllib.request.Request(cap_url, headers={"User-Agent": "Mozilla/5.0"})
    with urllib.request.urlopen(req) as response, open(cap_path, 'wb') as out_file:
        out_file.write(response.read())
    print(f"Downloaded {cap_path}")
else:
    print(f"File {cap_path} already exists.")

File c:\Users\shayo\OneDrive\Documents\ImageRetrieval-CNN-BERT\data\captions.json already exists.


In [79]:
if os.path.exists(cap_path):
    with open(cap_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print("Preview of captions.json content:")
    print(json.dumps(data, indent=2)[:300])

Preview of captions.json content:
[
  {
    "filename": "1.png",
    "caption": [
      "\u09a4\u09bf\u09a8 \u099c\u09a8 \u09ae\u09c7\u09df\u09c7 \u09ae\u09be\u09a8\u09c1\u09b7 \u0986\u099b\u09c7\u0964 \u098f\u0995 \u099c\u09a8 \u09a6\u09be\u09dc\u09bf\u09df\u09c7 \u0986\u099b\u09c7 \u0986\u09b0 \u09a6\u09c1\u0987 \u099c\u09a8 \u09a


In [80]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shayo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shayo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [81]:
# Helper functions

def preprocess(text):
    REPLACE_BY_SPACE_RE = re.compile(r'[/(){}\[\]\|@,;]')
    SYMBOLS_RE = re.compile(r'[^0-9a-z #+_]')
    NUMBERS = re.compile(r'\d+')
    STOPWORDS = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = SYMBOLS_RE.sub('', text)
    text = NUMBERS.sub('', text)

    tokens = [word for word in text.split() if word not in STOPWORDS]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens = " ".join(tokens)
    tokens = "" if not tokens else tokens
    
    return tokens

def batch_translate(captions, delimiter="|||"):
    if not captions:
        raise ValueError("Caption batch is empty.")

    valid_captions = [caption for caption in captions if caption]
    
    if not valid_captions:
        raise ValueError("All captions in the batch are empty or None.")

    try:
        concatenated = f" {delimiter} ".join(captions)
        translated = ts.translate_text(concatenated, translator='google', from_language='bn', to_language='en')

        if not translated:
            raise ValueError("Translation returned no results.")
        time.sleep(2)
        return translated.split(delimiter)
    except Exception as e:
        print(f"Error during translation: {e}")
        return [""] * len(captions)

In [82]:
if os.path.exists(cap_path):
    new_cap_path = os.path.join(data_folder, "new_captions.json")
    with open(cap_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    batch_size = 50
    all_captions = [entry['caption'][1] for entry in data]
    filenames = [entry['filename'] for entry in data]
    for i in tqdm(range(0, len(all_captions), batch_size), desc="Processing batches"):
        batch_captions = all_captions[i:i + batch_size]
        batch_filenames = filenames[i:i + batch_size]

        translated_captions = batch_translate(batch_captions)
        random_captions = batch_translate([random.choice(all_captions) for _ in range(len(batch_captions))])

        for filename, translated_caption, random_caption in zip(batch_filenames, translated_captions, random_captions):
            for entry in data:
                if entry['filename'] == filename:
                    entry['eng_caption'] = preprocess(translated_caption)
                    entry['false_caption'] = preprocess(random_caption)
                    break

    with open(new_cap_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print(f"Translated captions saved to {new_cap_path}")
else:
    print(f"File {cap_path} not found.")

Processing batches:   0%|          | 0/184 [00:00<?, ?it/s]

Error during translation: 


Processing batches: 100%|██████████| 184/184 [18:21<00:00,  5.98s/it]

Translated captions saved to c:\Users\shayo\OneDrive\Documents\ImageRetrieval-CNN-BERT\data\new_captions.json
